# NumPy 데이터 로드

이 튜토리얼은 NumPy 배열에서 tf.data.Dataset으로 데이터를 로드하는 예제를 제공합니다.

이 예제에서는 .npz 파일에서 MNIST 데이터세트를 로드합니다. 그러나 NumPy 배열의 소스는 중요하지 않습니다.


## 설정

In [2]:
import numpy as np
import tensorflow as tf

### .npz 파일에서 로드하기

In [3]:
DATA_URL = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'

path = tf.keras.utils.get_file('mnist.npz', DATA_URL)
with np.load(path) as data:
  train_examples = data['x_train']
  train_labels = data['y_train']
  test_examples = data['x_test']
  test_labels = data['y_test']

## `tf.data.Dataset`를 사용하여 NumPy 배열 로드하기

예제 배열과 레이블의 해당 배열이 있다고 가정하면, `tf.data.Dataset.from_tensor_slices`에 튜플로 두 배열을 전달하여 `tf.data.Dataset`을 만듭니다.

In [4]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))

## 데이터세트 사용하기

### 데이터세트 셔플 및 배치

In [5]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

### 모델 빌드 및 훈련

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_accuracy'])

In [7]:
model.fit(train_dataset, epochs=10)

Epoch 1/10


InternalError:  Blas GEMM launch failed : a.shape=(64, 784), b.shape=(784, 128), m=64, n=128, k=784
	 [[node sequential/dense/MatMul (defined at <ipython-input-7-ee484d448bfb>:1) ]] [Op:__inference_train_function_564]

Function call stack:
train_function


In [8]:
model.evaluate(test_dataset)

157/157 [==============================] - 0s 2ms/step - loss: 0.7410 - sparse_categorical_accuracy: 0.9558


[0.7409695386886597, 0.9557999968528748]